# Data Preprocessing Template

## Importing the libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

2024-03-13 10:01:12.266182: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 10:01:12.269972: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 10:01:12.443219: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 10:01:12.958095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 10:01:14.406950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Encode categorical data

In [3]:
dataset = pd.read_csv('chat_dataset.csv')
y = dataset.iloc[:, -1].values
X = dataset.iloc[:, :-1].values


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

dataset.iloc[:, -1] = y_encoded

dataset.to_csv('chat_dataset_encoded.csv', index=False)


## Split csv's into training set, test set and validation set

In [5]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('chat_dataset_encoded.csv')

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(len(train_df), len(test_df), len(validation_df))

train_df.to_csv('train_dataset.csv', index=False)
validation_df.to_csv('validation_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

408 88 88


## Importing the dataset

In [47]:
train_dataset = tf.data.experimental.make_csv_dataset(
    'train_dataset.csv',
    batch_size=50,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    shuffle=True,
    ignore_errors=True)

validation_dataset = tf.data.experimental.make_csv_dataset(
    'validation_dataset.csv',
    batch_size=10,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)

test_dataset = tf.data.experimental.make_csv_dataset(
    'test_dataset.csv',
    batch_size=10,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)


## Construct Tensorflow Hub embedding model

In [49]:
import tensorflow_hub as hub


embedding = "https://www.kaggle.com/models/google/nnlm/frameworks/TensorFlow2/variations/en-dim128-with-normalization/versions/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
